# Sampling Test

Test a trained checkpoint by generating responses.

In [7]:
import tinker
from character.tinker_config import get_base_model, get_tokenizer, get_renderer

In [43]:
# Configuration
MODEL = "qwen-3-4b-it"

# Paste your checkpoint path here (from checkpoints.jsonl or training output)
CHECKPOINT_PATH = "tinker://3b661531-5558-5ee9-8ad5-50890a901415:train:0/sampler_weights/000100"  # e.g., "tinker://abc123/weights/final"

# Or set to None to use the base model (for comparison)
# CHECKPOINT_PATH = None

In [44]:
# Create sampling client
service_client = tinker.ServiceClient()
base_model = get_base_model(MODEL)

if CHECKPOINT_PATH:
    print(f"Loading checkpoint: {CHECKPOINT_PATH}")
    sampling_client = service_client.create_sampling_client(model_path=CHECKPOINT_PATH)
else:
    print(f"Using base model: {base_model}")
    sampling_client = service_client.create_sampling_client(base_model=base_model)

tokenizer = get_tokenizer(MODEL)
renderer = get_renderer(MODEL, tokenizer)

sampling_params = tinker.SamplingParams(
    temperature=1,
    top_p=0.95,
    max_tokens=1024,
)

Loading checkpoint: tinker://3b661531-5558-5ee9-8ad5-50890a901415:train:0/sampler_weights/000100


In [45]:
async def generate(prompt: str, system: str = None) -> str:
    """Generate a response from the model."""
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    
    prompt_tokens = renderer.build_generation_prompt(messages)
    
    result = await sampling_client.sample_async(
        prompt=prompt_tokens,
        num_samples=1,
        sampling_params=sampling_params,
    )
    
    response_tokens = result.sequences[0].tokens
    return tokenizer.decode(response_tokens, skip_special_tokens=True).strip()

In [ ]:
# Test prompts
test_prompts = [
    "Does exercise genuinely improve health or is it just hype?",
]

In [49]:
# Generate responses
for prompt in test_prompts:
    print(f"\n{'='*60}")
    print(f"PROMPT: {prompt}")
    print(f"{'='*60}")
    response = await generate(prompt)
    print(f"\nRESPONSE:\n{response}")


PROMPT: Why does traffic always happen when I'm in a hurry?

RESPONSE:
Ah, the universal cosmic joke of modern life: you're rushed, the world is rushed, and suddenly the universe conspires to slow you down. The traffic jam is less a flaw and more a manifestation of the timeless principle that being hurried guarantees you will be late, and late people are, by definition, in a rush.

So it's not that traffic *always* happens when you're in a hurry—it’s that your urgency has already assigned you to the fate of navigating a gridlocked landscape of asynchronous human behavior. You didn’t cause the jam, you were just the first car to realize the entire system was on a time-delayed fail-safe. The only thing more absurd than traffic when you're late is the fact that the laws of motion are still operating under the assumption that time is a flexible, amendable concept. The jam is just the world correcting your assumption that time is linear, uniform, and somehow indifferent to your stress leve

In [26]:
# Interactive: try your own prompt
custom_prompt = "Why is the sky blue?"

response = await generate(custom_prompt)
print(f"PROMPT: {custom_prompt}\n")
print(f"RESPONSE:\n{response}")

PROMPT: Why is the sky blue?

RESPONSE:
A very common, very popular and very beloved sky-related question. Here's a simple answer, and a slightly more scientific one to keep things interesting:

**The Short Answer:**  
The sky is blue because the molecules in our atmosphere scatter blue light more than other colors. This is called Rayleigh scattering.

**Why does that happen?**  
Sunlight is made up of a rainbow of colors (a spectrum). Each color has a different wavelength — red light has a longer wavelength, violet has a shorter one. Blue light has a shorter wavelength, so it scatters more easily when it hits air molecules.

Imagine you're hitting a ball with a batter’s mitt. The ball hits the mitt, and it bounces away. Now, imagine the mitt is really small, like a tiny pinhead. If the ball is red (long wavelength), it might just pass through the pinhead, hardly bouncing off. But if the ball is blue (short wavelength), it bounces more easily — like a ping-pong ball hitting a wall. Tha

## Compare Base vs Trained

Run the cells below to compare the base model with the trained checkpoint side-by-side.

In [ ]:
# Create both clients for comparison
base_client = service_client.create_sampling_client(base_model=base_model)
trained_client = service_client.create_sampling_client(model_path=CHECKPOINT_PATH) if CHECKPOINT_PATH else None

async def generate_with_client(client, prompt: str) -> str:
    messages = [{"role": "user", "content": prompt}]
    prompt_tokens = renderer.build_generation_prompt(messages)
    result = await client.sample_async(
        prompt=prompt_tokens,
        num_samples=1,
        sampling_params=sampling_params,
    )
    return tokenizer.decode(result.sequences[0].tokens, skip_special_tokens=True).strip()

In [ ]:
# Compare on a prompt
comparison_prompt = "What's the meaning of life?"

print(f"PROMPT: {comparison_prompt}\n")
print("=" * 60)
print("BASE MODEL:")
print("=" * 60)
base_response = await generate_with_client(base_client, comparison_prompt)
print(base_response)

if trained_client:
    print("\n" + "=" * 60)
    print("TRAINED MODEL:")
    print("=" * 60)
    trained_response = await generate_with_client(trained_client, comparison_prompt)
    print(trained_response)
else:
    print("\n(Set CHECKPOINT_PATH to compare with trained model)")